Below are two code snippets. Both achieve the same thing, but the second
is arguably cleaner.

In [ ]:
list_of_words = ["This", "is", "a", "list", "of", "words", ".", "", "They",
                 "are", "separated", "into", "sentences", "by", "empty",
                 "strings", ".", "", "We", "want", "to", "turn", "this",
                 "into", "a", "list", "of", "sentences", "instead."]

In [ ]:
# option 1
list_of_sentences = [[]]
sentence_ind = 0
for word in list_of_words:
    if word != '':
        list_of_sentences[sentence_ind].append(word)
    else:
        list_of_sentences.append([])
        sentence_ind += 1

print(list_of_sentences)

In [ ]:
# option 2
list_of_sentences = []
sentence = []
for word in list_of_words:
    if word == "":  # I prefer to think in terms of equalities rather than inequalities
        list_of_sentences.append(sentence)
        sentence = []
    else:
        sentence.append(word)
list_of_sentences.append(sentence)  # because there's no empty string at the end

print(list_of_sentences)

Here's another, more complex example. The convoluted-vs.-clear distinction 
is really apparent in my opinion. :D
The problem: We have a list of pairs (tuples) of (shape, color). We want to
create conditional probability distributions for colors given shapes. This
can be represented conveniently as a dict of dicts. Then, an entry such as
dict[shape][color] would be the probability of the color given the shape.

In [ ]:
shapes_colors = [("circle", "red"), ("triangle", "red"), ("triangle", "blue"), ("circle", "red"),
                 ("triangle", "blue"), ("circle", "blue"), ("triangle", "red"), ("circle", "blue"),
                 ("circle", "red"), ("circle", "red")]

In [ ]:
# have fun understanding this one...
def dict_pair_prob(pair_list):
    list_length = len(pair_list)  # don't define a variable like this if you'll only use it once
    pair_list.sort()  # this is the point where I'd get suspicious -- you don't need to sort to count, usually
    pair_dict = {pair_list[0][0]: {}}
    counter = 1
    for idx in range(list_length - 1):  # if there was more between this and the definition of list_length, I might have forgotten what list this refers to -- use len(pair_list) directly!
        if pair_list[idx] == pair_list[idx+1]:
            counter += 1
        else:
            pair_dict[pair_list[idx][0]][pair_list[idx][1]] = counter  # way too many indices, hard to remember what these things actually are
            counter = 1
            if pair_list[idx][0] != pair_list[idx+1][0]:
                pair_dict[pair_list[idx+1][0]] = {}
    pair_dict[pair_list[-1][0]][pair_list[-1][1]] = counter

    # this second part is fine
    for freq_dict in pair_dict.values():
        total_freq = sum(freq_dict.values())
        for key, value in freq_dict.items():
            freq_dict[key] = value/total_freq
    return pair_dict

print(dict_pair_prob(shapes_colors))

In case you're wondering, the above code does the following:
1) Sort the list of pairs, by the first element then the second. I.e.
   all the circles come first, then all the triangles.
   Within a shape, first come all the blue ones, then the red ones.
2) Go through the list while the pair entries stay the same. Increase
   a counter by 1 for each entry.
3) If an element of the pair changes, create an entry in the dictionary
   with this counter. Start a new counter, go back to 2) unless the list
   is over.
4) (This is okay) Normalize to turn counts into probabilities

This is a very convoluted, unnecessarily complex way of solving this rather
simple counting problem. But not only that; understanding what is being done
is made far more difficult by not-so-descriptive variable names and excessive
use of indexing.
Below is a far clearer version that does the same thing.

In [ ]:
from collections import defaultdict

def dict_pair_prob_better(pair_list):
    pair_dict = {}
    for shape, color in pair_list:
        if shape not in pair_dict:
            pair_dict[shape] = defaultdict(float)  # use float because eventually that's what's gonna be in there
        pair_dict[shape][color] += 1

    for freq_dict in pair_dict.values():
        total_freq = sum(freq_dict.values())
        for key, value in freq_dict.items():
            freq_dict[key] = value/total_freq
    return pair_dict

print(dict_pair_prob_better(shapes_colors))

In [ ]:
# we can even use a defaultdict of defaultdicts to make the code shorter
def dict_pair_prob_short(pair_list):
    pair_dict = defaultdict(lambda: defaultdict(float))
    for shape, color in pair_list:
        pair_dict[shape][color] += 1

    for freq_dict in pair_dict.values():
        total_freq = sum(freq_dict.values())
        for key, value in freq_dict.items():
            freq_dict[key] = value/total_freq
    return pair_dict

print(dict_pair_prob_better(shapes_colors))